# AIML*** Assignment 
---********* Gaofeng Shi

In [1]:
# importing pandas, and other necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn import preprocessing
from pandas_profiling import ProfileReport
from warnings import filterwarnings
filterwarnings('ignore')
import seaborn as sns
import scipy.stats as stats
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from numpy import isnan
import re


In [2]:
# generate the data correlation heatmap for  data
def corr_heatmap(dtset):
    plt.figure(figsize=(16, 6))
    mask = np.triu(np.ones_like(dtset.corr(), dtype=np.bool))
    heatmap = sns.heatmap(dtset.corr(),mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG',xticklabels=True)
    heatmap.set_title('Data Correlation Heatmap', fontdict={'fontsize':18}, pad=12);
    heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=35) 
    # plt.savefig('alternative.png', dpi=300, bbox_inches='tight')
    return

In [3]:
#sns.pairplot(data)
def pair_plot(dtset):
    sns.set(style="ticks", color_codes=True)
    cols_to_plot = dtset.columns[7:17].tolist() 
    pairplot = sns.pairplot(dtset[cols_to_plot],hue="music_genre", palette="mako_r")
    #3plt.savefig('alternative.png', dpi=300, bbox_inches='tight')
    plt.show()
    return

In [4]:
# Load and proprocess the missing value by KNNimputer. process the numinal feature. 
def load_proprocess(dataName):
    dataname = "comp309-2021/" + dataName
    missing_values = ["?"]    # mark the missing values.
    dataset = pd.read_csv(dataname, na_values = missing_values)
    dataset['duration_ms'].replace(-1, np.nan, inplace = True)
    
    # feature 'mode' convert to numeric
    dataset['mode'] = dataset['mode'].map({'Major': 1, 'Minor': 0})
   
    # imputer the missing values with 3nn neighbors
    alte0 = dataset.select_dtypes(exclude='number')
    alte1 = dataset.select_dtypes(include='number')
    
    alte = alte0.join(alte1)
    
    # decomposted feature "track_hash" this is a irrelevened feature. redundent. 
    """
    alte['track0'] = alte['track_hash'].astype(str).str[0]
    chars = alte['track_hash'].astype(str).str[1]
    alte['track1'] = [ord(char) - 96 for char in chars]
    """
    alte.drop('track_hash', inplace=True, axis=1)
    
    # process the nominal features .
    alte['trc_n'] = alte['track_name'].apply(len)
    import re
    alte['trc_n1'] = [len(re.sub("[^0-9]", "", char)) for char in alte['track_name'].astype(str)]
    alte['trc_n2'] = [len(re.sub("[$0-9a-zA-Z .,:'/\()-_]", "", char)) for char in alte['track_name'].astype(str)]
    
    # get the track name of fist three words
    alte['firstT1'] = [ ord(char) for char in alte['track_name'].astype(str).str[0]]
    alte['lastT'] = alte['track_name'].str.split(' ').str[1]
    alte['lastT'] = [ord(char) for char in alte['lastT'].astype(str).str[0]]
    alte['lastT1'] = alte['track_name'].str.split(' ').str[2]
    alte['lastT1'] = [ord(char) for char in alte['lastT1'].astype(str).str[0]]
    alte['lastT2'] = alte['track_name'].str.split(' ').str[3]
    alte['lastT2'] = [ord(char) for char in alte['lastT2'].astype(str).str[0]]
    alte['lastT3'] = alte['track_name'].str.split(' ').str[4]
    alte['lastT3'] = [ord(char) for char in alte['lastT3'].astype(str).str[0]]
    
    return alte, alte0   

In [5]:
# load all the training data
anim, anim0 = load_proprocess('anime.csv')
alte, alte0 = load_proprocess('alternative.csv')
blue, blue0 = load_proprocess("blues.csv")
clas, clas0 = load_proprocess('classical.csv')
coun, coun0 = load_proprocess('country.csv')
elec, elec0 = load_proprocess('electronic.csv')
hiph, hiph0 = load_proprocess('hip-hop.csv')
jazz, jazz0 = load_proprocess('jazz.csv')
rap, rap0 = load_proprocess('rap.csv')
rock, rock0 = load_proprocess('rock.csv')


In [6]:
# append the data for training 
trainset = alte.append(anim)
trainset = trainset.append(clas)
trainset = trainset.append(blue)
trainset = trainset.append(coun)
trainset = trainset.append(elec)
trainset = trainset.append(hiph)
trainset = trainset.append(jazz)
trainset = trainset.append(rap)
trainset = trainset.append(rock)
train_y = trainset["music_genre"]

from sklearn.preprocessing import OrdinalEncoder
encoder_key = OrdinalEncoder() #handle_unknown = 'use_encoded_value', unknown_value=np.nan)
trainset['key'] = encoder_key.fit_transform(trainset['key'].values.reshape(-1, 1))
encoder_obd = OrdinalEncoder()
trainset['obtained_date'] = encoder_obd.fit_transform(trainset['obtained_date'].values.reshape(-1, 1))

# generate the laset column be class label : trainset_X
cols = list(trainset.columns.values)
cols.pop(cols.index("music_genre"))
cols.append("music_genre")
trainset_X = trainset[cols]
# pair_plot(trainset)
trainset_X.to_csv('trainset.csv',index=False)

# save processed data to csv 
blue.to_csv('blue.csv',index=False)
anim.to_csv('anim.csv',index=False)
alte.to_csv('alte.csv',index=False)
clas.to_csv('clas.csv',index=False)
coun.to_csv('coun.csv',index=False)
elec.to_csv('elec.csv',index=False)
hiph.to_csv('hiph.csv',index=False)
jazz.to_csv('jzaa.csv',index=False)
rap.to_csv('rap.csv',index=False)
rock.to_csv('rock.csv',index=False)

In [7]:
# process the test data
testset, test0 = load_proprocess('testing-instances.csv')
testset['key'] = encoder_key.transform(testset['key'].values.reshape(-1, 1))
testset['obtained_date'] = encoder_obd.transform(testset['obtained_date'].values.reshape(-1, 1))
test = testset

In [8]:
train_X = trainset
train_X.drop(['track_name', 'instance_id','music_genre'], inplace=True, axis=1)

### light GBM parameters. 
boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=318, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=1,
              bagging_fraction=1.0, feature_fraction=1.0, max_bin=255

In [9]:
# model the data 
from sklearn.datasets import make_classification
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier
from sklearn.preprocessing import LabelEncoder
train_X1 = train_X
i='artist_name'
le = LabelEncoder()
train_X1[i] = le.fit_transform(trainset[i])

#Set classes in test which don't exist in the encoder to 9999
test.loc[~test[i].isin(le.classes_),i] = 9999    
#Encode classes that exist in the encoder
test.loc[test[i].isin(le.classes_),i] = le.transform(test[i][test[i].isin(le.classes_)])

#Test the artist name data using KNN. 
train_X2=train_X1[[i]]
X_train, X_test, y_train, y_test = train_test_split(train_X2, train_y,test_size=0.30, random_state=43)

pipe1 = make_pipeline(RandomForestClassifier()) #StandardScaler(), 
pipe1.fit(X_train, y_train)  # apply scaling on training data Pipeline
pipe1.score(X_test, y_test) 

0.7354

In [10]:
# spilt test data to two dataset. add construction new feature
test_X1 = test
test_X1[i] = test[i].astype(int)
test1 = test.loc[test[i] != 9999]
test2 = test.loc[test[i] == 9999]

In [11]:
# Make prediction of test1 data atist name
test1[i] = test1[i].astype(int)
test2[i] = test2[i].astype(int)

pipe1.fit(train_X2, train_y)
test11 = test1[[i]]
test1_y = pipe1.predict(test11) # pipeline new featuer for test
train_k = pipe1.predict(train_X2) # pipeline new featuer for train

In [12]:
# reverse the label encode to 0-9 category
lecls = LabelEncoder()
lecls.fit(train_y)
test1['artist_name']=lecls.transform(test1_y).astype(int)
train_X['artist_name']=lecls.transform(train_k).astype(int)

In [13]:
# Set the none labeled art name to nan and Merge the test data. 
#test2['artist_name'].replace(9999, np.nan, inplace = True)
test1 = test1.append(test2)
result_id = test1[['instance_id']].astype(int)
test11 = test1

In [14]:
test11.drop(['track_name', 'instance_id'], inplace=True, axis=1)

In [15]:
# Imputer the training missing value with MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
#print("The number of missing values in the train data before imputation :\n", train_X.isnull().sum())
imputer_mice = IterativeImputer(max_iter=5, estimator
                                =DecisionTreeRegressor(),random_state=18)
train_X_mice = imputer_mice.fit_transform(train_X)
#The output is 'numpy.ndarray' so we convert it to dataframe for consistency
train_X_mice=pd.DataFrame(train_X_mice, columns=train_X.columns)
#print("The number of missing values :\n", train_X_mice.isnull().sum())

In [16]:
# Imputer the test missing value 
#print("The number of missing values in the test data before imputation :\n", test11.isnull().sum())
test1_mice = imputer_mice.transform(test11)
test1_mice=pd.DataFrame(test1_mice, columns=test11.columns)
#print("The number of missing values in the test data after imputation :\n", test1_mice.isnull().sum())

In [17]:
train_X.head()

,artist_name,key,obtained_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,...,tempo,valence,trc_n,trc_n1,trc_n2,firstT1,lastT,lastT1,lastT2,lastT3
0,0,9.0,3.0,48,0.35900,0.423,428800.0,0.738,0.084000,0.0801,...,111.414,0.276,17,0,0,76,89,116,68,110
1,6,7.0,3.0,46,0.29900,0.733,221787.0,0.444,0.000003,0.1040,...,89.106,0.372,7,0,0,80,110,110,110,110
2,0,10.0,3.0,43,0.29700,0.666,262560.0,0.740,0.000000,0.1450,...,124.911,0.620,18,0,0,78,76,89,68,110
3,0,4.0,3.0,49,0.02360,0.561,191947.0,0.764,0.000014,0.2090,...,90.004,0.657,13,0,0,72,87,110,110,110
4,0,9.0,3.0,43,0.00972,0.652,226333.0,0.813,0.000041,0.2900,...,124.000,0.560,13,0,0,87,89,65,110,110


In [18]:
# Light GBM test the final data. with default parameters value
# Test whole dataset using LGBMclassfification
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,test_size=0.30, random_state=43)
lgbmc=LGBMClassifier()
pipe2 = make_pipeline(LGBMClassifier()) #StandardScaler(), 
pipe2.fit(X_train, y_train)  # apply scaling on training data Pipeline
acc=pipe2.score(X_test, y_test) 
print("The training accuracy is: ", acc )

The training accuracy is:  0.826


In [19]:
test1_mice.tail()

,artist_name,key,obtained_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,...,tempo,valence,trc_n,trc_n1,trc_n2,firstT1,lastT,lastT1,lastT2,lastT3
19995,9999.0,4.0,3.0,48.0,0.08860,0.846,240533.0,0.832,0.2000,0.0737,...,126.132,0.780,72.0,0.0,0.0,77.0,82.0,40.0,45.0,40.0
19996,9999.0,2.0,3.0,59.0,0.00235,0.624,225253.0,0.599,0.0967,0.2180,...,79.982,0.657,6.0,0.0,0.0,66.0,110.0,110.0,110.0,110.0
19997,9999.0,10.0,3.0,36.0,0.96000,0.398,462547.0,0.273,0.8810,0.1030,...,131.667,0.114,20.0,0.0,0.0,87.0,84.0,82.0,71.0,110.0
19998,9999.0,11.0,3.0,45.0,0.49600,0.465,504867.0,0.644,0.1790,0.1760,...,81.894,0.355,13.0,0.0,0.0,72.0,65.0,66.0,110.0,110.0
19999,9999.0,11.0,3.0,52.0,0.29300,0.801,215973.0,0.740,0.0000,0.3690,...,92.042,0.828,29.0,0.0,0.0,87.0,67.0,84.0,87.0,45.0


In [20]:
test1_fill_9999 = test1_mice.loc[test1_mice[i] != 9999]
test2_fill_9999 = test1_mice.loc[test1_mice[i] == 9999]

In [21]:
train_X_fill_9999 = train_X
train_X_fill_9999.drop('artist_name', inplace=True, axis=1)

In [22]:
test2_copy = test2_fill_9999
test2_copy.drop('artist_name', inplace=True, axis=1)


In [23]:
# make preddiction test2 artist_name
pipe2.fit(train_X_fill_9999, train_y)
test2_artist_name = pipe2.predict(test2_copy)

In [24]:
test2_fill_9999['artist_name']=lecls.transform(test2_artist_name).astype(int)
test2_fill_9999.tail()

,key,obtained_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,...,valence,trc_n,trc_n1,trc_n2,firstT1,lastT,lastT1,lastT2,lastT3,artist_name
19995,4.0,3.0,48.0,0.08860,0.846,240533.0,0.832,0.2000,0.0737,-5.486,...,0.780,72.0,0.0,0.0,77.0,82.0,40.0,45.0,40.0,6
19996,2.0,3.0,59.0,0.00235,0.624,225253.0,0.599,0.0967,0.2180,-7.303,...,0.657,6.0,0.0,0.0,66.0,110.0,110.0,110.0,110.0,9
19997,10.0,3.0,36.0,0.96000,0.398,462547.0,0.273,0.8810,0.1030,-18.115,...,0.114,20.0,0.0,0.0,87.0,84.0,82.0,71.0,110.0,7
19998,11.0,3.0,45.0,0.49600,0.465,504867.0,0.644,0.1790,0.1760,-7.646,...,0.355,13.0,0.0,0.0,72.0,65.0,66.0,110.0,110.0,7
19999,11.0,3.0,52.0,0.29300,0.801,215973.0,0.740,0.0000,0.3690,-7.162,...,0.828,29.0,0.0,0.0,87.0,67.0,84.0,87.0,45.0,6


In [25]:
test_X = test1_fill_9999.append(test2_fill_9999)
test_X.tail()

,artist_name,key,obtained_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,...,tempo,valence,trc_n,trc_n1,trc_n2,firstT1,lastT,lastT1,lastT2,lastT3
19995,6.0,4.0,3.0,48.0,0.08860,0.846,240533.0,0.832,0.2000,0.0737,...,126.132,0.780,72.0,0.0,0.0,77.0,82.0,40.0,45.0,40.0
19996,9.0,2.0,3.0,59.0,0.00235,0.624,225253.0,0.599,0.0967,0.2180,...,79.982,0.657,6.0,0.0,0.0,66.0,110.0,110.0,110.0,110.0
19997,7.0,10.0,3.0,36.0,0.96000,0.398,462547.0,0.273,0.8810,0.1030,...,131.667,0.114,20.0,0.0,0.0,87.0,84.0,82.0,71.0,110.0
19998,7.0,11.0,3.0,45.0,0.49600,0.465,504867.0,0.644,0.1790,0.1760,...,81.894,0.355,13.0,0.0,0.0,72.0,65.0,66.0,110.0,110.0
19999,6.0,11.0,3.0,52.0,0.29300,0.801,215973.0,0.740,0.0000,0.3690,...,92.042,0.828,29.0,0.0,0.0,87.0,67.0,84.0,87.0,45.0


In [26]:
# make preddiction 
pipe2.fit(train_X_mice, train_y)
test_y = pipe2.predict(test_X)
test_y

array(['Anime', 'Electronic', 'Classical', ..., 'Jazz', 'Jazz', 'Hip-Hop'],
      dtype=object)

In [27]:
# save the result.
result = result_id
result['music_genre'] = test_y.tolist()
result.to_csv('result1109.0.csv',index=False)
result

,instance_id,music_genre
0,1,Anime
1,2,Electronic
2,3,Classical
3,4,Classical
4,5,Alternative
...,...,...
19885,19886,Hip-Hop
19893,19894,Rock
19920,19921,Jazz
19959,19960,Jazz


# ---------------------------------------------------------------------------------
# As an end line for the main programming.
## rest of are the testing code